# This notebook does the following:
## 1. Read DL2 file
## 2. Use the config file to generate the event list HDUs
## 3. Add the IRF to create a DL3
## 4. Index the DL3 files in the given folder

In [1]:
import numpy as np
from pathlib import Path

from traitlets.config.loader import Config

from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u

In [2]:
from lstchain.io.io import read_data_dl2_to_QTable
from lstchain.reco.utils import get_effective_time
from lstchain.paths import run_info_from_filename, dl2_to_dl3_filename
# Post merger of PR #752
from lstchain.irf.hdu_table import (
    create_event_list, add_icrs_position_params,
    create_hdu_index_hdu, create_obs_index_hdu
)
from lstchain.io.event_selection import EventSelector, DL3FixedCuts
from lstchain.io.config import read_configuration_file

# 1. Input parameters to convert DL2 file to DL3

In [22]:
base_dir = "/fefs/aswg/data/real/DL2/"
# Directory path with observation date, data production and tailcut cleaning
dl2_dir = "20201120/v0.7.5_test_new_calibration_tree/tailcut84_dynamic_cleaning/"

input_dl2_path = base_dir + dl2_dir + "dl2_LST-1.Run02976.h5"
output_dl3_path = "./DL3/"
Path(output_dl3_path).mkdir(exist_ok=True)

config_file = output_dl3_path + "irf_tool_config.json"
irf_file = output_dl3_path + "pnt_irf_pnt_gamma.fits.gz"

source_name = "Crab"
source_ra = "83.63308333deg"
source_dec = "22.0145deg"

overwrite = True

In [23]:
dl3_filename = dl2_to_dl3_filename(input_dl2_path)
source_pos = SkyCoord(ra=source_ra, dec=source_dec)
print("The name of the DL3 file will be,", dl3_filename)
print("Using the source RA and DEC values to get the source position,", source_pos)

The name of the DL3 file will be, dl3_LST-1.Run02976.fits.gz
Using the source RA and DEC values to get the source position, <SkyCoord (ICRS): (ra, dec) in deg
    (83.63308333, 22.0145)>


# 2. Get the information of the data DL2 file

In [24]:
%%time
data = read_data_dl2_to_QTable(input_dl2_path)

CPU times: user 8.49 s, sys: 8.75 s, total: 17.2 s
Wall time: 19.2 s


In [25]:
%%time
effective_time, elapsed_time = get_effective_time(data)
run_number = run_info_from_filename(input_dl2_path)[1]
print("The effective time of the run is,",effective_time, "and the total elapsed time of the run is,", elapsed_time)
print("The run number used from the DL2 file is,", run_number)

The effective time of the run is, 1109.6082797617908 s and the total elapsed time of the run is, 1159.4717936515808 s
The run number used from the DL2 file is, 2976
CPU times: user 54.2 ms, sys: 11.1 ms, total: 65.3 ms
Wall time: 61 ms


# 3. Apply selection cuts and bin the events as per the config file

In [26]:
config = Config(read_configuration_file(config_file))
config

{'EventSelector': {'filters': {'intensity': [0, inf],
   'width': [0, inf],
   'length': [0, inf],
   'r': [0, 1],
   'wl': [0.1, 1],
   'leakage_intensity_width_2': [0, 1],
   'event_type': [32, 33]}},
 'DL3FixedCuts': {'fixed_gh_cut': 0.7,
  'fixed_gh_max_efficiency': 0.9,
  'fixed_theta_cut': 0.2,
  'allowed_tels': [1]},
 'DataBinning': {'true_energy_min': 0.01,
  'true_energy_max': 100,
  'true_energy_n_bins_per_decade': 5.5,
  'reco_energy_min': 0.01,
  'reco_energy_max': 100,
  'reco_energy_n_bins_per_decade': 5.5,
  'energy_migration_min': 0.2,
  'energy_migration_max': 5,
  'energy_migration_n_bins': 31,
  'fov_offset_min': 0.1,
  'fov_offset_max': 1.1,
  'fov_offset_n_edges': 9,
  'bkg_fov_offset_min': 0,
  'bkg_fov_offset_max': 10,
  'bkg_fov_offset_n_edges': 21,
  'source_offset_min': 0.0001,
  'source_offset_max': 1.0001,
  'source_offset_n_edges': 1000}}

In [27]:
# Using the Components for the event selection and for applying some cuts
event_sel = EventSelector(config=Config(config))
fixed_cuts = DL3FixedCuts(config=Config(config))

In [28]:
%%time
# Applying event filters to the Hillas parameters
print("Size of the table before the filter -",len(data))
data = event_sel.filter_cut(data)
print("Size of the table after the filter -",len(data))

Size of the table before the filter - 4032878
Size of the table after the filter - 3863530
CPU times: user 1.7 s, sys: 1.59 s, total: 3.28 s
Wall time: 3.14 s


In [29]:
%%time
# Applying the gammaness cut
print("Size of the table before the cut -",len(data))
data = fixed_cuts.gh_cut(data)
print("Size of the table after the cut -",len(data))

Size of the table before the cut - 3863530
Size of the table after the cut - 44143
CPU times: user 207 ms, sys: 167 ms, total: 375 ms
Wall time: 371 ms


# 4. Convert the position parameters of the events in ICRS frame and add to the table

In [30]:
%%time
data = add_icrs_position_params(data, source_pos)

CPU times: user 172 ms, sys: 7.3 ms, total: 179 ms
Wall time: 176 ms


# 5. Create the HDUs for the DL3 file

In [31]:
%%time
events, gti, pointing, data_params = create_event_list(
    data, run_number, source_name, source_pos, effective_time.value, elapsed_time.value
)

CPU times: user 114 ms, sys: 2.49 ms, total: 116 ms
Wall time: 113 ms


In [32]:
# Parameters that can be used for IRF interpolation target values
data_params

{'ZEN_PNT': <Quantity 15.9 deg>,
 'AZ_PNT': <Quantity 247.6 deg>,
 'B_DELTA': <Quantity 61.1 deg>}

# 6. Add the HDUs to the final HDUList

In [33]:
hdulist = fits.HDUList([fits.PrimaryHDU(), events, gti, pointing])

In [34]:
irf_hdus = fits.open(irf_file)

In [35]:
for irf in irf_hdus[1:]:
    hdulist.append(irf)

# 7. Write the HDUList to the final DL3 file

In [36]:
hdulist.writeto(output_dl3_path + dl3_filename, overwrite=overwrite)

# 8. Check the DL3 fits info

In [37]:
f = fits.open(output_dl3_path + dl3_filename)
f.info()

Filename: ./DL3/dl3_LST-1.Run02976.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  EVENTS        1 BinTableHDU     76   44143R x 11C   [K, D, D, D, D, D, K, D, D, D, D]   
  2  GTI           1 BinTableHDU     29   1R x 2C   [D, D]   
  3  POINTING      1 BinTableHDU     44   1R x 5C   [D, D, D, D, D]   
  4  EFFECTIVE AREA    1 BinTableHDU     50   1R x 5C   [21D, 21D, D, D, 21D]   
  5  ENERGY DISPERSION    1 BinTableHDU     56   1R x 7C   [21D, 21D, 30D, 30D, D, D, 630D]   
  6  BACKGROUND    1 BinTableHDU     50   1R x 5C   [21D, 21D, 20D, 20D, 420D]   


# 9. Index the DL3 files in the folder

In [38]:
%%time
# Get the list of all DL3 files in the given output destination
list_dl3_files = sorted(Path(output_dl3_path).glob("dl3*gz"))
file_list = []
for d in list_dl3_files:
    file_list.append(d.name)
print(file_list)

['dl3_LST-1.Run02976.fits.gz', 'dl3_LST-1.Run02977.fits.gz']
CPU times: user 483 µs, sys: 1.38 ms, total: 1.86 ms
Wall time: 1.14 ms


In [39]:
%%time
create_hdu_index_hdu(
    file_list, 
    Path(output_dl3_path), 
    Path(output_dl3_path)/"hdu-index.fits.gz", 
    overwrite
)

Run 2976 does not contain HDU PSF
Run 2976 does not contain HDU GH CUTS
Run 2976 does not contain HDU RAD_MAX
Run 2977 does not contain HDU PSF
Run 2977 does not contain HDU GH CUTS
Run 2977 does not contain HDU RAD_MAX


CPU times: user 89 ms, sys: 8.66 ms, total: 97.6 ms
Wall time: 95.1 ms


In [40]:
%%time
create_obs_index_hdu(
    file_list, 
    Path(output_dl3_path), 
    Path(output_dl3_path)/"obs-index.fits.gz", 
    overwrite
)

CPU times: user 101 ms, sys: 2.12 ms, total: 103 ms
Wall time: 101 ms
